<a href="https://colab.research.google.com/github/Shavrau/AVA1-AM-Spotify/blob/main/artistas_escalonamento_nomeado_spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importação de bibliotecas

In [44]:
!pip install spotipy pandas matplotlib

In [45]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import matplotlib.pyplot as plt
import requests
import base64

#Carregamento do dataset

In [46]:
CLIENT_ID = '5d6396e8ea294d0982be0aa32d871bf7'
CLIENT_SECRET = '959fbe6295a9493cac1be0a1c4f6aa6c'

def autenticar_spotify():
    auth_manager = SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET
    )
    return spotipy.Spotify(auth_manager=auth_manager)

sp = autenticar_spotify()

In [47]:
def obter_dados_artistas(ids_artistas):
    artistas = sp.artists(ids_artistas)['artists']
    dados = []

    for artista in artistas:
        dados.append({
            'artist_id': artista['id'],
            'artist_name': artista['name'],
            'followers': artista['followers']['total'],
            'popularity': artista['popularity'],
            'genres': ', '.join(artista['genres']),
            'num_genres': len(artista['genres'])
        })

    return pd.DataFrame(dados)

In [48]:
def obter_dados_em_lote(ids_artistas, tamanho_lote=50):
    dados = []
    for i in range(0, len(ids_artistas), tamanho_lote):
        lote = ids_artistas[i:i + tamanho_lote]
        dados.extend(obter_dados_artistas(lote).to_dict('records'))
    return pd.DataFrame(dados)

In [49]:
def buscar_ids_artistas_por_genero(genero, limite=50):
    resultados = sp.search(q=f'genre:{genero}', type='artist', limit=limite)
    return [artista['id'] for artista in resultados['artists']['items']]

In [50]:
def buscar_ids_por_generos(generos, limite_por_genero=50):
    ids = []
    for genero in generos:
        try:
            resultados = buscar_ids_artistas_por_genero(genero, limite=limite_por_genero)
            ids.extend(resultados)
        except Exception as e:
            print(f"Erro ao buscar artistas do gênero '{genero}': {e}")
    return list(set(ids))


In [57]:
generos = [
    'acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime',
    'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat',
    'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical',
    'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house',
    'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
    'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage',
    'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy',
    'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays',
    'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial',
    'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids',
    'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore',
    'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera',
    'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep',
    'power-pop', 'progressive-house', 'psych-rock', 'punk', 'punk-rock',
    'r-n-b', 'rainy-day', 'reggae', 'reggaeton', 'road-trip', 'rock', 'rock-n-roll',
    'rockabilly', 'romance', 'sad', 'salsa', 'samba', 'sertanejo', 'show-tunes',
    'singer-songwriter', 'ska', 'sleep', 'songwriter', 'soul', 'soundtracks',
    'spanish', 'study', 'summer', 'swedish', 'synth-pop', 'tango', 'techno',
    'trance', 'trip-hop', 'turkish', 'work-out', 'world-music'
]

ids_artistas = buscar_ids_por_generos(generos, limite_por_genero=50)
df_artistas = obter_dados_em_lote(ids_artistas)
print(f"Total de artistas coletados: {len(df_artistas)}")
df_artistas

Total de artistas coletados: 4315


,artist_id,artist_name,followers,popularity,genres,num_genres
0,5FQ8tBUtIamA2hRtatrYUF,Marc Seguí,203109,68,,0
1,46beGPwcX2QEZcmuvrMqp4,Juan D'Arienzo,51036,42,tango,1
2,5VVN3xZw1i2qihfITZlvCZ,G. V. Prakash,14367543,79,"tamil pop, kollywood, tamil dance, tamil hip h...",5
3,4mrBetqy378Jf1y6NLszlx,VINAI,501739,67,"melbourne bounce, slap house, big room",3
4,4HIOOPiZCJm2MPRU8Cs4MK,Los 4,616223,58,"timba, salsa",2
...,...,...,...,...,...,...
4310,5sehJiCiH15ihXYCKyEALX,Manikka Vinayagam,242774,57,"tamil dance, tamil pop, kollywood",3
4311,7E7RsGtRMYVH8UDRWLBofH,Bunny Hop Dance Party,779,8,,0
4312,0ABk515kENDyATUdpCKVfW,Westside Gunn,755667,69,"boom bap, alternative hip hop, jazz rap",3
4313,05pAwLhsutiuj6gerEwGvU,Lefty Frizzell,115809,41,"honky tonk, traditional country",2


#Exploração Inicial

In [52]:
df_artistas.describe()

,followers,popularity,num_genres
count,4.317000e+03,4317.000000,4317.000000
mean,2.240781e+06,57.460042,2.327311
std,7.225005e+06,16.699855,1.789457
min,0.000000e+00,0.000000,0.000000
25%,6.914800e+04,50.000000,1.000000
50%,3.763980e+05,59.000000,2.000000
75%,1.579412e+06,68.000000,3.000000
max,1.362684e+08,100.000000,12.000000


In [53]:
df_artistas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4317 entries, 0 to 4316
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_id    4317 non-null   object
 1   artist_name  4317 non-null   object
 2   followers    4317 non-null   int64 
 3   popularity   4317 non-null   int64 
 4   genres       4317 non-null   object
 5   num_genres   4317 non-null   int64 
dtypes: int64(3), object(3)
memory usage: 202.5+ KB


In [54]:
df_artistas.isnull().sum()

,0
artist_id,0
artist_name,0
followers,0
popularity,0
genres,0
num_genres,0


In [55]:
(df_artistas['genres'].str.strip() == '').sum()

np.int64(695)